# Week 9: Bisector Area of 3D Mesh
**Target**: Fit a plane with fixed normal that can bisector the surface area of any given mesh

## Table of Contents
- [0 - Packages](#0)
    - [0.1 - Packages](#0-1)
    - [0.2 - Self-defined Functions](#0-2)
- [1 - Implementation](#1)
    - [1.1 - Problem Description](#1-1)
    - [1.2 - Methodology](#1-2)
    - [1.3 - The Code](#1-3)
- [2 - Forward Propagation](#2)
    - [2.1 - Area Calculation](#2-1)
- [3 - Backward Propagation](#3)
    - [3.1 - Gradients of Cross Product](#3-1)
    - [3.2 - Gradients of L2 Norm](#3-2)
- [4 - Examples](#4)
    - [4.1 - Sphere](#4-1)
    - [4.2 - Standford Bunny](#4-2)
    - [4.3 - ](#4-2)
- [5 - Reference](#5)

<a name='0'></a>
## 0 - Package and Tools
<a name='0-1'></a>
### 0.1 - Packages

In [1]:
# Packages
import numpy as np
import random
# Visualization
import pyvista as pv
from pyvista import examples

<a name='0-2'></a>
### 0.2 - Self-defined Functions

In [2]:
# Self-defined functions
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
from util.mesh.triangle.R3 import make_clockwise_3d

<a name='1'></a>
## 1 - Implementation

<a name='1-1'></a>
### 1.1 - Problem Description

To formalize the problem, I define the problem to be **finding a plane with fixed normal $N = (A, B, C)$ that can bisector the surface area of any given mesh in three dimentional space**. As a result, the plane is represented as $Ax+By+Cz+D=0$, where $D$ is the variable that I try to optimize.

<a name='1-2'></a>
### 1.2 - Methodology

I use gradient descent to find the optimal value of $D$. 

#### 1.2.1 - Initial Solution
I start by calculating the barycenter of the mesh:

$$G=\frac{\sum_{i=1}^{n} v}{n}$$

where $v$ is the coordinate of the vertices. Then I generate the initial $D$ by making the fixed-normal plane passing through the barycenter, which can be done by

#### 1.2.2 - The Loss

To monitor the optimization, I use the squre of difference between the target ratio of area $S_{target}$, which can be pre-calculated, and the area $S$ the enclosed by my decision boundary. In this case will be the sum area of all triangles and parts of triangles that at the direction of plane's normal.

$$L=(S-S_{target})^2$$

<a name='1-3'></a>
### 1.3 - The code

This is the implementation. The following parts will go through details of the implementation concerning some of the crucial parts.

In [3]:
class problem_mesh_3D:
    def __init__(self, points, faces, normals, n=[0,1,0], ratio=0.5, learning_rate=0.05, num_iteration=1000):
        if len(points) < 3:
            A = np.array([6,9])
            B = np.array([10,14])
            C = np.array([8,28])
            self.vertices = np.array([A, B, C])
        else:
            self.vertices = points
        self.faces = faces
        self.normals = normals
        util_3d.make_clockwise_3d(self.vertices)
        
        # Initialize variable "D" and define the plane
        self.D = 0.5
        self.plane=np.array(n + [self.D])
        self.N=n
    
        # Area realted
        self.ratio = ratio
        self.compute_mesh_area()
        
        # Learning realted
        self.lr = learning_rate
        self.num_iteration = num_iteration
        
    def compute_mesh_area(self):
        self.area = 0
        for face in self.faces:
            index_A, index_B, index_C = face
            A = self.vertices[index_A]
            B = self.vertices[index_B]
            C = self.vertices[index_C]
            self.area += util.triangle_area(A, B, C)
        return self.area
    
    def forward_propagate(self):
        self.S = 0
        self.dS_dD = 0
        
        for idx, face in enumerate(self.faces):
            index_A, index_B, index_C = face
            A = self.vertices[index_A]
            B = self.vertices[index_B]
            C = self.vertices[index_C]
            
            # Edges
            line_AC = util_3d.line_3D(A, C-A)
            line_AB = util_3d.line_3D(A, B-A)
            line_BA = util_3d.line_3D(B, A-B)
            line_BC = util_3d.line_3D(B, C-B)
            line_CB = util_3d.line_3D(C, B-C)
            line_CA = util_3d.line_3D(C, C-A)
            
            SA=np.array([0,0,0])
            SB=np.array([0,0,0])
            SC=np.array([0,0,0])
            dSA_dD=np.array([0,0,0])
            dSB_dD=np.array([0,0,0])
            dSC_dD=np.array([0,0,0])
            
            if (np.dot(self.N, A) + self.D) > 0:
                if not np.isclose(np.dot(line_AC.n, self.N), 0) and not np.isclose(np.dot(line_AB.n, self.N), 0):
                    # The area
                    AC = line_AC.intersection_plane_t(self.plane) * line_AC.n
                    AB = line_AB.intersection_plane_t(self.plane) * line_AB.n
                    # Directed area
                    SA = np.cross(AC, AB) / 2
                    # The gradients
                    # dt/dD
                    dAC_dD = line_AC.n / np.dot(line_AC.n, self.N)
                    dAB_dD = line_AB.n / np.dot(line_AB.n, self.N)
                    dSA_dD = dv.derivative_cross_product(AC, AB, dAC_dD, dAB_dD) / 2
                
            if (np.dot(self.N, B) + self.D) > 0:
                if not np.isclose(np.dot(line_BA.n, self.N), 0) and not np.isclose(np.dot(line_BC.n, self.N), 0):
                    # The area
                    BA = line_BA.intersection_plane_t(self.plane) * line_BA.n
                    BC = line_BC.intersection_plane_t(self.plane) * line_BC.n
                    # Directed area
                    SB = np.cross(BA, BC) / 2
                    # The gradients
                    # dt/dD
                    dBA_dD = line_BA.n / np.dot(line_BA.n, self.N)
                    dBC_dD = line_BC.n / np.dot(line_BC.n, self.N)
                    dSB_dD = dv.derivative_cross_product(BA, BC, dBA_dD, dBC_dD) / 2
                    
            if (np.dot(self.N, C) + self.D) > 0:
                if not np.isclose(np.dot(line_CB.n, self.N), 0) and not np.isclose(np.dot(line_CA.n, self.N), 0):
                    # The area
                    CB = line_CB.intersection_plane_t(self.plane) * line_CB.n
                    CA = line_CA.intersection_plane_t(self.plane) * line_CA.n
                    # Directed area
                    SC = np.cross(CB, CA) / 2
                    # The gradients
                    # dt/dD
                    dCB_dD = line_BA.n / np.dot(line_CB.n, self.N)
                    dCA_dD = line_BC.n / np.dot(line_CA.n, self.N)
                    dSC_dD = dv.derivative_cross_product(CB, CA, dCB_dD, dCA_dD) / 2

            self.S += np.linalg.norm(SA + SB + SC)
            self.dS_dD += dv.derivative_L2_norm(-(SA + SB + SC), dSA_dD+dSB_dD+dSC_dD)
    
    def loss_MSE(self):
        return np.power(self.S-self.ratio*self.area, 2)
    
    def backward_propagate(self):
        dL_dS = 2*self.S-2*self.ratio*self.area
        print("dL/dS: ", dL_dS)
        self.dL_dD=dL_dS*self.dS_dD
        
    def update_parameters(self):
        A, B, C, D = self.plane
        self.pre_D = D
        D -= self.lr * self.dL_dD
        self.D = D
        self.plane = np.array([A, B, C, D])
        
    def visualize(self):
        A, B, C, D = self.plane
#         plane_pre = pv.Plane(center=(0, -self.pre_D, 0), direction=(0, 1, 0))
        plane = pv.Plane(center=(0, -self.D, 0), direction=(0, 1, 0), i_size=2, j_size=2, i_resolution=1, j_resolution=1)
        
        plotter = pv.Plotter()
#         plotter.add_mesh(mesh.points[0], color="red", show_edges=True)
        plotter.add_mesh(mesh, color="lightblue", show_edges=True)
        plotter.add_mesh(plane, color="yellow", show_edges=True)
#         plotter.add_mesh(plane_pre, color="orange", show_edges=True)
        plotter.view_vector(vector=normal, viewup=[0, 1, 0])
        plotter.show_axes()
        plotter.show()
        
    def train_one_round(self):
        self.forward_propagate()
        print("The loss is: ", self.loss_MSE())
        self.backward_propagate()
        self.update_parameters()
        
        print("The enclosing area is: ", self.S)
        print("Half area： ", self.ratio*self.area)
        print("dL/dD is: ", self.dL_dD)
        print("D is: ", self.plane[3])
        
    def train(self):
        for i in range(self.num_iteration):
            self.forward_propagate()
            self.backward_propagate()
            self.update_parameters()
        
        print("The loss is: ", self.loss_MSE())
        print("The enclosing area is: ", self.S)

<a name='2'></a>
## 2 - Forward Propagation

<a name='2-1'></a>
### 2.1 - Area Calculation

To obtain the correct enclosed by the decision boundary, I apply **directed area** and takes the magnitude.

#### 2.1.1 - Local Analysis



<a name='3'></a>
## 3. Backward Propagation

When calculating the gradients at each round, it is important to know $\frac{dS_i}{dD}$. As I mentioned above, the enclosing area of each triangle is calculated by directed area

<a name='3-1'></a>
### 3.1 Gradients of Cross Product

As it is mention above, the directed area is calculated by the cross product of two edge of subdivided triangles, supposing to be $\vec{AR}$ and $\vec{AP}$ ($P$ lies on line $AB$ and $R$ lies on $AC$). To obtain the correct gradients of directed area, I apply the conclusion [[1]](#ref1):

$$d(a \times b) = da \times b + a \times db$$

Specifically, in our case

$$\frac{dS_{ARP}}{dD}=\frac{d(\vec{AR} \times \vec{AP})}{dD}$$

This works similarly as $\frac{dS_{BPQ}}{dD}$ and $\frac{dS_{CQR}}{dD}$. Finally, the gradient of directed area gives:

$$\frac{dS_{dir}}{dD}=\frac{dS_{ARP}}{dD}+\frac{dS_{BPQ}}{dD}+\frac{dS_{CQR}}{dD}$$

The code is:

In [4]:
def derivative_cross_product(A, B, dA, dB):
    return np.cross(dA, B) + np.cross(A, dB)

<a name='3-2'></a>
### 3.2 Gradients of L2 Norm

As the directed area is a vector, but the desired area is a scalar. So we have to take the magnitude of the directed area. This gives the gradients of each components:

$$\frac{d||v||_2}{dv_i} = \frac{1}{2} * (v1^2 + v2^2 + ... + vn^2)^{(- \frac{1}{2})} * 2*v_i = \frac{v_i} {||v||_2}$$

In our case, components of directed area $dS_{dir}$ is a function about variable $D$. So:

$$\frac{dS_i}{dD}=\frac{dS_{dir}^x}{dD}+\frac{dS_{dir}^y}{dD}+\frac{dS_{dir}^z}{dD}$$

The code is:

In [5]:
def derivative_L2_norm(v, dv_dt):
    v_norm = np.linalg.norm(v)
    if v_norm == 0:
        return 0
    else:
        df_dt = np.dot(v, dv_dt) / v_norm
        return df_dt

<a name='4'></a>
## 4 - Examples

In [6]:
points = np.array([[0, 1, 1], [-1, -1, -1], [1, 0.5, 0], [-2,-3,-4]])
faces = np.array([3, 0, 1, 2, 3, 0, 3, 1])
make_clockwise_3d(points)
mesh = pv.PolyData(points, faces)

plane = pv.Plane(center=(0, 0.5, 0), direction=(0, 1, 0))

center = mesh.points.mean(axis=0)
normal = mesh.face_normals[0]

plotter = pv.Plotter()
plotter.add_mesh(mesh, color="lightblue", show_edges=True)
plotter.add_mesh(plane, color="yellow", show_edges=True)
plotter.view_vector(vector=normal, viewup=[0, 1, 0])
plotter.show_axes()
plotter.show()


--------------------------------------------------------------------------------
   !!! You are currently using trame@3 which may break your application !!!
--------------------------------------------------------------------------------

 1. trame@3 only provides by default trame.widgets.[html,client] and remove
    everything else as implicit dependency. Those other widgets will still
    exist and will be supported, but they will need to be defined as a
    dependency of your application.

       $ pip install trame-vtk trame-vuetify trame-plotly

    Import paths are remaining the same.

    For libraries like vuetify since they offer different API between
    their vue2 and vue3 implementation, the widget name will reflect
    which vue version they are referencing. But original naming will remain.

       from trame.widgets import vuetify2, vuetify3


 2. trame@3 aims to use vue3 as a new default. But to smooth the transition
    we will maintain the server.client_type = 'vue2' 

Widget(value="<iframe src='http://localhost:63273/index.html?ui=P_0x14ee891e820_0&reconnect=auto' style='width…

In [484]:
test = problem_mesh_3D(sphere.points, sphere.faces.reshape((-1,4))[:, 1:4], sphere.face_normals)

In [35]:
test = problem_mesh_3D(mesh.points, mesh.faces.reshape((-1,4))[:, 1:4], mesh.face_normals)

In [54]:
test.train_one_round()

The loss is:  0.0
dL/dS:  0.0
The enclosing area is:  1.566799212912266
Half area：  1.566799212912266
dL/dD is:  0.0
D is:  0.05223396330896816


In [55]:
test.visualize()


Widget(value="<iframe src='http://localhost:50245/index.html?ui=P_0x1770a694b10_2&reconnect=auto' style='width…

In [29]:
sphere = pv.Sphere()

pl = pv.Plotter()
pl.add_mesh(sphere, show_edges=True, color="white")
pl.add_points(sphere.points, color="red", point_size=5)
pl.show_axes()
pl.show()

Widget(value="<iframe src='http://localhost:61327/index.html?ui=P_0x2ac0fd6ee50_3&reconnect=auto' style='width…